# Star Schema Lab

### Introduction

In this lesson, we'll see some of the benefits of using the star schema.  Our context is to use a database of movies, this time constructed with the star schema.  Let's get started.   

### Exploring our Database

We can begin by conneccting to our database.

In [3]:
import sqlite3
conn = sqlite3.connect('star_films.db')
cursor = conn.cursor()

In [4]:
cursor.execute('SELECT name from sqlite_master where type= "table"')
cursor.fetchall()

[('release_dates',), ('writers',), ('directors',), ('actors',), ('movies',)]

So we have: 

* fact table of `movies` and,
* dimension tables of `release_dates`, `writers`, `directors`, and `actors`.

Now let's begin by taking a look at the fact table.

In [5]:
cursor.execute('PRAGMA table_info(movies)')
cursor.fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'title', 'TEXT', 0, None, 0),
 (2, 'runtime', 'REAL', 0, None, 0),
 (3, 'release_date_id', 'INTEGER', 0, None, 0),
 (4, 'popularity', 'TEXT', 0, None, 0),
 (5, 'vote_average', 'REAL', 0, None, 0),
 (6, 'vote_count', 'REAL', 0, None, 0),
 (7, 'revenue', 'REAL', 0, None, 0),
 (8, 'budget', 'TEXT', 0, None, 0),
 (9, 'profit', 'REAL', 0, None, 0)]

As we can see, except for the title, every attribute follows our rule of being either a metric -- runtime, popularity, revenue, budget and profit -- or a foreign key, like `release_date_id`.  Notice that already our table is not normalized: profit is a derivative of revenue and budget.  But, we are ok with this in our OLAP, where we sacrifice single source of truth for queries that are faster and easier to write. 

Now let's look at a diagram of our entire schema.

<img src="./starred_films.png" width="60%">

The largest change we made when moving from a OLTP database, to our OLAP database here, was to collapse our many to many relationships of writers, actors, and directors into a belongs to has many relationship.  So, for example, even though an actor does have many movies, our star schema only represents the relationship that a movie has many actors.  This is because we anticipate our queries will largely answer questions about our fact table of *movie*, and not an actor. 

> Notice, that with the above schema, we still *can* construct the query to find all of the movies a specific actor is in, we'll just have to specify the actor's name instead of any actor id.  For example: 
> ```sql
SELECT *.movies FROM movies JOIN actors on movies.actor_id = actors.id WHERE actors.name = 'Harrison Ford';
```

### Constructing Queries

Ok, now we'll learn more about the benefit of constructing queries with the star schema in the next lesson, but let's get some preliminary practice.

Once again here is our schema.

> <img src="./starred_films.png" width="50%">

Answer the following questions:

1. What are the top five movies by profit.

In [1]:
import pandas as pd

In [23]:
pd.read_sql('SELECT movies.* FROM movies ORDER BY profit DESC LIMIT 5', conn)

# 	id	title	runtime	release_date_id	popularity	vote_average	vote_count	revenue	budget	profit
# 0	1089	Avatar	162.0	432	185.070892	7.2	12114.0	2.787965e+09	237000000	2.550965e+09
# 1	405	Titanic	98.0	5599	26.88907	7.5	7770.0	1.845034e+09	200000000	1.645034e+09
# 2	15109	Titanic	194.0	5603	26.88907	7.5	7770.0	1.845034e+09	200000000	1.645034e+09
# 3	2294	Jurassic World	123.0	2421	32.790475	6.5	8842.0	1.513529e+09	150000000	1.363529e+09
# 4	317	Furious 7	140.0	1752	27.275687	7.3	4253.0	1.506249e+09	190000000	1.316249e+09

,id,title,runtime,release_date_id,popularity,vote_average,vote_count,revenue,budget,profit
0,1089,Avatar,162.0,432,185.070892,7.2,12114.0,2.787965e+09,237000000,2.550965e+09
1,405,Titanic,98.0,5599,26.88907,7.5,7770.0,1.845034e+09,200000000,1.645034e+09
2,15109,Titanic,194.0,5603,26.88907,7.5,7770.0,1.845034e+09,200000000,1.645034e+09
3,2294,Jurassic World,123.0,2421,32.790475,6.5,8842.0,1.513529e+09,150000000,1.363529e+09
4,317,Furious 7,140.0,1752,27.275687,7.3,4253.0,1.506249e+09,190000000,1.316249e+09


Next let's find the top five actors whose movies have the highest total profit.

In [45]:
pd.read_sql('''SELECT actors.name, SUM(movies.profit) as total_profit FROM movies JOIN actors ON actors.movie_id = movies.id
GROUP BY actors.name ORDER BY total_profit DESC LIMIT 5''', conn)

# 	name	total_profit
# 0	Ian McKellen	6.409442e+09
# 1	Michelle Rodriguez	6.366621e+09
# 2	Tom Hanks	6.070977e+09
# 3	Samuel L. Jackson	5.716205e+09
# 4	Dwayne Johnson	5.640074e+09

,name,total_profit
0,Ian McKellen,6.409442e+09
1,Michelle Rodriguez,6.366621e+09
2,Tom Hanks,6.070977e+09
3,Samuel L. Jackson,5.716205e+09
4,Dwayne Johnson,5.640074e+09


Now let's find the directors with the largest total budget.

In [43]:
pd.read_sql('''SELECT directors.name, SUM(movies.budget) as total_budget FROM movies JOIN directors ON directors.movie_id = movies.id
GROUP BY directors.name ORDER BY total_budget DESC LIMIT 5''', conn)

# name	avg_budget
# 0	Michael Bay	1620000000
# 1	Steven Spielberg	1618000000
# 2	Ridley Scott	1423000000
# 3	Peter Jackson	1116672000
# 4	Christopher Nolan	1105006000

,name,total_budget
0,Michael Bay,1620000000
1,Steven Spielberg,1618000000
2,Ridley Scott,1423000000
3,Peter Jackson,1116672000
4,Christopher Nolan,1105006000


And let's find the writers with whose total `popularity` score is in the top ten, as well as the count of the number of movies for each writer.

In [46]:
pd.read_sql('''SELECT writers.name, COUNT(movies.id) as num_movies, SUM(movies.popularity) as total_popularity FROM movies JOIN writers ON writers.movie_id = movies.id
GROUP BY writers.name ORDER BY total_popularity DESC LIMIT 5''', conn)

# 	name	num_movies	total_popularity
# 0	Brian Lynch	3	567.406736
# 1	Edgar Wright	7	326.178462
# 2	Christopher Nolan	10	320.734508
# 3	James Cameron	8	310.277008
# 4	Michael Jelenic	1	294.337037

,name,num_movies,total_popularity
0,Brian Lynch,3,567.406736
1,Edgar Wright,7,326.178462
2,Christopher Nolan,10,320.734508
3,James Cameron,8,310.277008
4,Michael Jelenic,1,294.337037


### Summary

In this lesson, we practiced working with the star schema.  First, we saw the structure of our schema, and how it differs from an OLTP schema.  Our OLAP schema took the following form.

> <img src="./starred_films.png" width="50%">

As we saw, we collapsed our many to many relationships which simplified our schema and reduced the number of joins we would need to perform a query.  We can see that our fact table contained data about the movie, and foreign keys to dimension tables.  

Then we performed queries -- many of which that used a group by to analyze the movies of different according to different attributes -- like the director, writer, or actor.